In [120]:
!pip install contractions
!pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 2.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=1eb24ec7d2bcd7c2f40846c3ebfcbd4413463e8909cc1611c22d51cc117ea433
  Stored in directory: /Users/ankush.singal/Library/Caches/pip/wheels/ab/0f/23/3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect


In [127]:
import pandas as pd
import contractions
from autocorrect import Speller

from tqdm.notebook import tqdm_notebook

#initial tqdm for pandas.apply functions
tqdm_notebook.pandas()

pdObj = pd.read_json('CasesWithLinksInComments.json', orient='index')
pdObj.to_csv('streaming.csv', index=True)


In [116]:
#https://stackoverflow.com/questions/43332057/pandas-strip-white-space
df = pd.read_csv('streaming.csv', converters={'Description': str.strip})
df.rename(
    columns=({'Unnamed: 0': 'Session_Id'}), 
    inplace=True,
)
df.replace(r'\s+',' ', regex = True, inplace = True) #remove spaces(/r/n in the description include whitespace)
df['exurl'] = df.Description.str.extract(r'(https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+)') #links within the description
df.head()

,Session_Id,attributes,Subject,Description,IsClosed,matches,exurl
0,5008Y00001uS4GMQA0,"{'type': 'Case', 'url': '/services/data/v47.0/...",[ARR] [Sev B] SR-2205300030001910 title: DBFS ...,The cx wants to upload files using DBFS API 2....,True,['https://docs.microsoft.com/en-us/azure/datab...,NaN
1,5008Y00001uSAYMQA4,"{'type': 'Case', 'url': '/services/data/v47.0/...",Few S3 folders were dropped in production,"Hi Team, On production in our freshworks-it-bu...",True,['https://docs.databricks.com/delta/delta-util...,NaN
2,5008Y00001uSEGLQA4,"{'type': 'Case', 'url': '/services/data/v47.0/...",Upload table issue,"Hello, We are testing to upload a table with a...",True,['https://docs.databricks.com/data/data-source...,https://dbc-af2bbc71-2647.cloud.databricks.com
3,5008Y00001uS7sxQAC,"{'type': 'Case', 'url': '/services/data/v47.0/...",spatial join not working,"Hi, We have a daily job that tag some events w...",True,['https://docs.databricks.com/delta/join-perfo...,https://dbc-f1098e44-02b2.cloud.databricks.com
4,5008Y00001uSFfqQAG,"{'type': 'Case', 'url': '/services/data/v47.0/...",Unable to import The installed R library,"Hello Team, We have successfully installed R l...",True,['https://kb.databricks.com/r/install-rjava-rj...,NaN


# LOWERING THE CASE

In [125]:
df['Subject'] = df['Subject'].apply(lambda x: x.lower())
df['Description'] = df['Description'].apply(lambda x: x.lower())

# REMOVING CONTRACTIONS

In [122]:
def cont_expand(text):
    """ Expand contractions(i.e. don't -> do not, can't -> cannot)"""
    expanded_text = contractions.fix(text,slang=False)
    return expanded_text

In [124]:
df['Subject'] = df['Subject'].progress_apply(cont_expand)
df['Description'] = df['Description'].progress_apply(cont_expand)

  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/362 [00:00<?, ?it/s]

# AUTOCORRECT MISSPELLED WORDS

In [130]:
spell_check = Speller(lang='en')
df['Subject'] = df['Subject'].progress_apply(lambda x: spell_check(str(x)))
df['Description'] = df['Description'].progress_apply(lambda x: spell_check(str(x)))

  0%|          | 0/362 [00:00<?, ?it/s]

  0%|          | 0/362 [00:00<?, ?it/s]

In [132]:
list_of_attr_urls = [dic["url"] for dic in df.attributes.to_list()]
df["url"] = list_of_attr_urls
df["type"] = [ dic["type"] for dic in df.attributes.to_list() ]
df.drop(columns=["attributes"],inplace=True)

TypeError: string indices must be integers